In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys  
sys.path.insert(0, r'C:\Users\Rodrigo\Desktop\PhD\Study1\Physiological_Data')

In [3]:
import warnings
from Signals_Processing import *
from Process import *
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import ensemble, feature_selection
import pickle

In [4]:
os.chdir('C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\')
os.getcwd()

'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system'

In [5]:
"""Load Data from Folder"""

folder = os.getcwd() + "\\Training Models\\"
participant = "P0_stimuli"
path = folder + participant
path

'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0_stimuli'

In [6]:
EEG_baseline = pd.read_csv(r'C:\Users\Rodrigo\Desktop\PhD\loop-system\Training Models\P0\EEGbaseline.csv', sep=";", index_col=0)

In [7]:
df_baseline = pd.read_csv(r'C:\Users\Rodrigo\Desktop\PhD\loop-system\Training Models\P0\baseline.csv', sep=";", index_col=0)

In [8]:
imp = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0\\imp.pkl", "rb"))
scaler = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0\\scaler.pkl", "rb"))
rfe = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0\\rfe.pkl", "rb"))
model = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0\\model.pkl", "rb"))

In [9]:
EEG_baseline

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,EEG_30_gamma,EEG_30_theta,EEG_31_alpha,EEG_31_betha,EEG_31_gamma,EEG_31_theta,EEG_32_alpha,EEG_32_betha,EEG_32_gamma,EEG_32_theta
0,-0.000034,0.000036,-0.000072,-0.00029,9.138866e-08,-2.380761e-07,-8.622854e-07,-0.000003,-0.000014,-0.000015,...,0.000138,0.000529,-2.747438e-07,-8.349964e-07,-0.000001,-0.000006,-3.129658e-07,-7.537620e-07,-4.429443e-07,-0.000003


In [10]:
df_baseline

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_CVSD,RRV_MedianBB,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1
0,79.730853,63.154386,96.768817,6.259845,0.757272,0.620034,0.950053,60.7714,44.7724,65,...,0.539439,332.0,57.8214,0.174161,39.0,23.0,51.111111,86.666667,0.067151,149.748344


In [14]:
os.chdir(path)
(
    users,
    EEG_epochs,
    EEG_filtered,
    Signals_epochs,
    Signals,
    EEG_dict,
    features_signals,
    features_EEG,
    features_epochs,
    features_epochs_EEG,
) = ({}, {}, {}, {}, {}, {}, {}, {}, {}, {})

for root, dirs, files in os.walk(path):
    for fname in files:
        if fname.endswith(".xdf"):
            users[fname] = Run_files(fname)

In [15]:
users

{'P0_stimuli.xdf': [{'info': defaultdict(list,
               {'name': ['Emteq'],
                'type': ['Emteq.StreamType'],
                'channel_count': ['9'],
                'channel_format': ['float32'],
                'source_id': [None],
                'nominal_srate': ['2000.000000000000'],
                'version': ['1.100000000000000'],
                'created_at': ['600295.2744091999'],
                'uid': ['8f5625d7-d7e0-41e0-9a90-73d8d36e2113'],
                'session_id': ['default'],
                'hostname': ['DESKTOP-8HVM7BE'],
                'v4address': [None],
                'v4data_port': ['16572'],
                'v4service_port': ['16572'],
                'v6address': [None],
                'v6data_port': ['16572'],
                'v6service_port': ['16572'],
                'desc': [defaultdict(list,
                             {'channels': [defaultdict(list,
                                           {'channel': [defaultdict(list,
      

In [16]:
Opensignals_fs = 100
EEG_fs = 250
resolution = 16

In [17]:
sensors = ["ECG", "EDA", "RESP"]
data = {}
for user in users.keys():
    data[user.split(".")[0]] = Load_Data(
        users[user],
        "OpenSignals",
        "openvibeSignal",
        "PsychoPy Markers",
        "PsychoPy Markers",
        sensors,
    )

    data[user.split(".")[0]]["Markers"].pop(0)
    data[user.split(".")[0]]["Markers"].pop(-1)
    data[user.split(".")[0]]["Markers Timestamps"].pop(0)
    data[user.split(".")[0]]["Markers Timestamps"].pop(-1)

onset, offset, videos, valence, arousal = ({}, {}, {}, {}, {})
for user in data.keys():
    valence[user] = data[user]["Valence"]
    arousal[user] = data[user]["Arousal"]
    onset[user], offset[user], videos[user] = getMarkers(
        data[user]["Markers"], data[user]["Markers Timestamps"]
    )

onset_index = {}
offset_index = {}
onset_index_EEG = {}
offset_index_EEG = {}

for user in data.keys():
    onset_index[user], offset_index[user] = getMarkersIndex(
        onset[user], offset[user], data[user]["Signals"]["Time"]
    )
    onset_index_EEG[user], offset_index_EEG[user] = getMarkersIndex(
        onset[user], offset[user], np.array(data[user]["EEG"]["Time"])
    )

events_diff = {}

for keys in onset.keys():
    events_diff[keys] = CalculateEventsDiff(onset[keys], offset[keys])

In [18]:
"""EEG Processing"""

EEG_filtered = filterEEG(data, EEG_fs)
EEG_filtered = getEEGChannels(EEG_filtered)
EEG_epochs = getEpochs(EEG_filtered, onset_index_EEG, events_diff, EEG_fs)
EEG_dict = getVideosDict(EEG_epochs, videos)
features_EEG = getEEGBands(EEG_dict, EEG_fs)

In [19]:
features_EEG

{'P0_stimuli': {'EMDB/Scenery/5009.avi': {'EEG_1': {'alpha': 2.605012259075971e-09,
    'betha': 2.2229779076183416e-09,
    'gamma': 1.2352267132838486e-09,
    'theta': 2.899633653046142e-08},
   'EEG_2': {'alpha': 2.5340226058349487e-10,
    'betha': 1.8449708581735515e-10,
    'gamma': 5.856435993033069e-11,
    'theta': 1.4259977527869127e-09},
   'EEG_3': {'alpha': 1.771486475576023e-07,
    'betha': 1.999041210878615e-07,
    'gamma': 8.733863147041717e-08,
    'theta': 2.6634402992782003e-07},
   'EEG_4': {'alpha': 8.766944169920008e-10,
    'betha': 6.838520063443022e-10,
    'gamma': 2.4254022631094447e-10,
    'theta': 3.015702435791737e-09},
   'EEG_5': {'alpha': 1.1349698068924443e-10,
    'betha': 8.304779425172623e-11,
    'gamma': 2.4731087181346485e-11,
    'theta': 2.5690537382906093e-10},
   'EEG_6': {'alpha': 2.731760318065026e-08,
    'betha': 4.157800844523352e-08,
    'gamma': 2.1520195322879377e-08,
    'theta': 4.2658353058568835e-08},
   'EEG_7': {'alpha': 2.9

In [20]:
EEG_list = EEG_baseline.loc[0].values.flatten().tolist()

In [21]:
warnings.filterwarnings("ignore")
Signals_epochs = getSignalsEpochs(data, onset_index, events_diff, Opensignals_fs)
Signals = getVideosDict(Signals_epochs, videos)
features_signals = getFeatures(Signals, Opensignals_fs, resolution)

In [22]:
columns = list()
category, video = [], []

for key in features_epochs_EEG.keys():
    user = key
    print(user)
videos = list(features_EEG[user].keys())
for epochs in features_EEG[user][videos[0]].keys():
    for bands in features_EEG[user][videos[0]][epochs].keys():
        columns.append(epochs + "_" + bands)
df_EEG = pd.DataFrame(columns=columns)

for users in features_EEG.keys():
    temp_df = pd.DataFrame(columns=columns)

    for i, videos in enumerate(features_EEG[users].keys()):
        if videos != "baseline":
            temp_list = list()
            for channel in features_EEG[users][videos].keys():
                for band in features_EEG[users][videos][channel].keys():
                    temp_list.append(
                        features_EEG[users][videos][channel][band]
                    )
            temp_df.loc[i] = list(np.array(temp_list)-np.array(EEG_list))
            category.append(videos.split("/")[1])
            video.append(videos.split("/")[2])

    df_EEG = pd.concat([df_EEG, temp_df], ignore_index=True)
df_EEG["Category"] = category
df_EEG["Video"] = video

In [23]:
df_EEG

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,EEG_31_alpha,EEG_31_betha,EEG_31_gamma,EEG_31_theta,EEG_32_alpha,EEG_32_betha,EEG_32_gamma,EEG_32_theta,Category,Video
0,0.000034,-0.000036,0.000072,0.00029,-9.113526e-08,2.382606e-07,8.623440e-07,0.000003,0.000014,0.000016,...,3.253068e-07,8.657558e-07,0.000001,0.000007,3.187565e-07,7.578014e-07,4.445379e-07,0.000003,Scenery,5009.avi
1,0.000034,-0.000036,0.000072,0.00029,-9.100867e-08,2.382413e-07,8.623355e-07,0.000003,0.000014,0.000015,...,3.296176e-07,8.643407e-07,0.000001,0.000007,3.203119e-07,7.573266e-07,4.444917e-07,0.000003,Scenery,5005.avi
2,0.000034,-0.000036,0.000072,0.00029,-9.110056e-08,2.382849e-07,8.623613e-07,0.000003,0.000014,0.000016,...,3.322906e-07,8.734822e-07,0.000001,0.000007,3.200817e-07,7.588324e-07,4.448736e-07,0.000003,Scenery,5000.avi
3,0.000034,-0.000036,0.000072,0.00029,-9.107638e-08,2.382632e-07,8.623388e-07,0.000003,0.000014,0.000015,...,3.323219e-07,8.736768e-07,0.000001,0.000007,3.206447e-07,7.578613e-07,4.445036e-07,0.000003,Scenery,5008.avi
4,0.000034,-0.000036,0.000072,0.00029,-9.100845e-08,2.385332e-07,8.626112e-07,0.000003,0.000014,0.000015,...,3.669544e-07,9.177997e-07,0.000001,0.000007,3.230385e-07,7.634360e-07,4.500698e-07,0.000003,Scenery,5006.avi
5,0.000034,-0.000036,0.000072,0.00029,-9.110492e-08,2.383687e-07,8.625174e-07,0.000003,0.000014,0.000015,...,3.259995e-07,8.794907e-07,0.000001,0.000007,3.171161e-07,7.570102e-07,4.447207e-07,0.000003,Horror,1006.avi
6,0.000034,-0.000036,0.000072,0.00029,-9.115283e-08,2.384314e-07,8.625891e-07,0.000003,0.000014,0.000015,...,3.128312e-07,8.741106e-07,0.000001,0.000006,3.172511e-07,7.571294e-07,4.449246e-07,0.000003,Horror,1008.avi
7,0.000034,-0.000036,0.000072,0.00029,-9.102621e-08,2.386342e-07,8.627569e-07,0.000003,0.000014,0.000015,...,3.312168e-07,8.862162e-07,0.000001,0.000006,3.194963e-07,7.583034e-07,4.450600e-07,0.000003,Horror,1003.avi
8,0.000034,-0.000036,0.000072,0.00029,-9.117393e-08,2.384431e-07,8.625304e-07,0.000003,0.000014,0.000015,...,3.106533e-07,8.756067e-07,0.000001,0.000006,3.172464e-07,7.575363e-07,4.453097e-07,0.000003,Horror,1000.avi
9,0.000034,-0.000036,0.000072,0.00029,-9.106578e-08,2.386966e-07,8.629239e-07,0.000003,0.000014,0.000016,...,3.304683e-07,8.911341e-07,0.000001,0.000006,3.183317e-07,7.577255e-07,4.451016e-07,0.000003,Horror,1009.avi


In [24]:
category, videos = [], []

for key in features_signals.keys():
    user = key
video = list(features_signals[user].keys())
df = pd.DataFrame(columns=features_signals[user][video[0]].columns)
for users in features_signals.keys():
    for epoch in features_signals[users].keys():
        if epoch != "baseline":
            df = pd.concat([df, features_signals[users][epoch].sub(df_baseline)], ignore_index=True)
            category.append(epoch.split("/")[1])
            videos.append(epoch.split("/")[2])
df["Category"] = category
df["Video"] = videos

In [25]:
df

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1,Category,Video
0,7.452809,6.595770,1.567468,1.207992,-0.063766,-0.009883,-0.089840,1.3795,-12.9167,-58,...,31.8759,0.106143,-33.0,-19.0,-1.111111,-11.666667,NaN,-60.761052,Scenery,5009.avi
1,-2.507750,-3.169161,-8.555251,1.751588,0.028755,0.060133,0.050194,26.9246,16.4191,-45,...,44.4780,0.117290,-35.0,-19.0,15.555556,-20.000000,NaN,145.547524,Scenery,5005.avi
2,10.071387,-64.661174,4.901038,7.043165,-0.784058,-40.439847,-0.189865,5164.4762,5266.6339,-58,...,9.6369,-0.054021,-35.0,-20.0,-1.111111,-20.000000,NaN,-11.320204,Scenery,5000.avi
3,-5.648013,-3.169161,-13.456004,-0.014514,0.058674,0.100143,0.050194,11.2317,20.1683,-42,...,28.1694,0.062728,-34.0,-20.0,-8.253968,-15.238095,NaN,-79.731680,Scenery,5008.avi
4,4.129796,-0.669791,8.468395,6.327130,-0.024869,-0.049894,0.010184,53.0574,15.0376,-49,...,16.3086,0.039779,-35.0,-19.0,15.555556,-20.000000,NaN,90.721820,Scenery,5006.avi
5,-3.312744,-3.763275,-7.238933,0.968810,0.035171,0.050133,0.060199,16.8725,5.2609,-52,...,62.2692,0.149533,-35.0,-19.0,28.888889,-6.666667,NaN,291.771757,Horror,1006.avi
6,-7.882336,-5.476465,-8.555527,2.071038,0.089076,0.060135,0.090207,36.9356,19.4924,-46,...,-40.0302,-0.120166,-36.0,-21.0,-17.777778,-36.666667,NaN,-76.576287,Horror,1008.avi
7,1.030334,-0.669864,1.567480,3.528489,-0.003046,-0.009883,0.010185,33.2294,7.2963,-40,...,99.3342,0.175074,-36.0,-20.0,23.888889,-11.666667,NaN,60.167512,Horror,1003.avi
8,-6.778717,-64.679171,-12.282849,6.073622,-0.802389,-39.969847,0.010187,5734.7181,5867.8706,-44,...,-47.4432,-0.143501,-38.0,-22.0,-38.611111,-74.166667,NaN,-128.590209,Horror,1000.avi
9,1.523790,2.046981,-3.041852,0.739162,-0.012938,0.020123,-0.029827,8.2392,7.0907,-53,...,-4.4478,-0.012667,-35.0,-20.0,-1.111111,-20.000000,NaN,45.682846,Horror,1009.avi


In [26]:
"""Concatenate Dataframes"""
columns = df.columns[: (len(df.columns) - 2)]
columns_EEG = df_EEG.columns[: (len(df_EEG.columns) - 2)]
full_dataframe = pd.concat([df_EEG[columns_EEG], df], axis=1)
full_columns = full_dataframe.columns[: (len(full_dataframe.columns)-2)]

In [27]:
full_dataframe

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1,Category,Video
0,0.000034,-0.000036,0.000072,0.00029,-9.113526e-08,2.382606e-07,8.623440e-07,0.000003,0.000014,0.000016,...,31.8759,0.106143,-33.0,-19.0,-1.111111,-11.666667,NaN,-60.761052,Scenery,5009.avi
1,0.000034,-0.000036,0.000072,0.00029,-9.100867e-08,2.382413e-07,8.623355e-07,0.000003,0.000014,0.000015,...,44.4780,0.117290,-35.0,-19.0,15.555556,-20.000000,NaN,145.547524,Scenery,5005.avi
2,0.000034,-0.000036,0.000072,0.00029,-9.110056e-08,2.382849e-07,8.623613e-07,0.000003,0.000014,0.000016,...,9.6369,-0.054021,-35.0,-20.0,-1.111111,-20.000000,NaN,-11.320204,Scenery,5000.avi
3,0.000034,-0.000036,0.000072,0.00029,-9.107638e-08,2.382632e-07,8.623388e-07,0.000003,0.000014,0.000015,...,28.1694,0.062728,-34.0,-20.0,-8.253968,-15.238095,NaN,-79.731680,Scenery,5008.avi
4,0.000034,-0.000036,0.000072,0.00029,-9.100845e-08,2.385332e-07,8.626112e-07,0.000003,0.000014,0.000015,...,16.3086,0.039779,-35.0,-19.0,15.555556,-20.000000,NaN,90.721820,Scenery,5006.avi
5,0.000034,-0.000036,0.000072,0.00029,-9.110492e-08,2.383687e-07,8.625174e-07,0.000003,0.000014,0.000015,...,62.2692,0.149533,-35.0,-19.0,28.888889,-6.666667,NaN,291.771757,Horror,1006.avi
6,0.000034,-0.000036,0.000072,0.00029,-9.115283e-08,2.384314e-07,8.625891e-07,0.000003,0.000014,0.000015,...,-40.0302,-0.120166,-36.0,-21.0,-17.777778,-36.666667,NaN,-76.576287,Horror,1008.avi
7,0.000034,-0.000036,0.000072,0.00029,-9.102621e-08,2.386342e-07,8.627569e-07,0.000003,0.000014,0.000015,...,99.3342,0.175074,-36.0,-20.0,23.888889,-11.666667,NaN,60.167512,Horror,1003.avi
8,0.000034,-0.000036,0.000072,0.00029,-9.117393e-08,2.384431e-07,8.625304e-07,0.000003,0.000014,0.000015,...,-47.4432,-0.143501,-38.0,-22.0,-38.611111,-74.166667,NaN,-128.590209,Horror,1000.avi
9,0.000034,-0.000036,0.000072,0.00029,-9.106578e-08,2.386966e-07,8.629239e-07,0.000003,0.000014,0.000016,...,-4.4478,-0.012667,-35.0,-20.0,-1.111111,-20.000000,NaN,45.682846,Horror,1009.avi


In [28]:
X = np.array(full_dataframe[full_columns])
Y = np.array(full_dataframe[["Category"]])

In [29]:
X = imp.transform(X)
X = scaler.transform(X)
X = rfe.transform(X)

prediction = model.predict(X)
print(prediction)
print(full_dataframe[["Category"]])
from sklearn import metrics
print("Accuracy = " + str(metrics.accuracy_score(np.array(full_dataframe[["Category"]]),prediction)*100)+"%")

['Scenery' 'Scenery' 'Scenery' 'Scenery' 'Scenery' 'Scenery' 'Scenery'
 'Scenery' 'Scenery' 'Scenery']
  Category
0  Scenery
1  Scenery
2  Scenery
3  Scenery
4  Scenery
5   Horror
6   Horror
7   Horror
8   Horror
9   Horror
Accuracy = 50.0%


In [11]:
output = pd.read_csv(r'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\P0\output_categories.csv', sep=";", index_col=False)
features = pd.read_csv(r'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\P0\final_features1.csv', sep=";", index_col=False)

In [12]:
output

,Variable,True Value,Prediction
0,Category,Scenery,Scenery
1,Category,Scenery,Scenery
2,Category,Scenery,Scenery
3,Category,Scenery,Scenery
4,Category,Scenery,Scenery
...,...,...,...
2341,Category,Horror,Scenery
2342,Category,Horror,Scenery
2343,Category,Horror,Scenery
2344,Category,Horror,Scenery


In [56]:
print("Real Accuracy = " + str(metrics.accuracy_score(np.array(output["True Value"]),np.array(output["Prediction"]))*100)+"%")

Real Accuracy = 46.71781756180734%


In [30]:
features

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,RRV_MedianBB,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1,True Value
0,0.000726,0.000554,0.000406,0.008257,0.001308,0.000955,0.000318,0.007591,0.020926,0.024346,...,-26.0,-1.4826,0.009953,-32.0,-19.0,-6.666667,-8.888889,NaN,-66.309744,Scenery
1,0.000754,0.000573,0.000415,0.008481,0.001362,0.000985,0.000327,0.007791,0.021085,0.024505,...,3.0,41.5128,0.122359,-33.0,-19.0,-6.666667,-20.000000,NaN,-67.468334,Scenery
2,0.000804,0.000624,0.000424,0.008551,0.001434,0.001058,0.000332,0.007861,0.020569,0.023598,...,3.0,41.5128,0.122359,-33.0,-19.0,-6.666667,-20.000000,NaN,-67.468334,Scenery
3,0.000829,0.000647,0.000435,0.008788,0.001475,0.001090,0.000341,0.008054,0.020905,0.024034,...,3.0,41.5128,0.122359,-33.0,-19.0,-6.666667,-20.000000,NaN,-67.468334,Scenery
4,0.000857,0.000670,0.000447,0.009058,0.001523,0.001126,0.000351,0.008295,0.021110,0.024291,...,3.0,41.5128,0.122359,-33.0,-19.0,-6.666667,-20.000000,NaN,-67.492700,Scenery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2342,0.000404,0.003670,0.005588,0.002055,0.000701,0.001830,0.002433,0.002822,0.020486,0.025910,...,98.0,108.9711,0.213729,-35.0,-21.0,-17.777778,-20.000000,NaN,3.668245,Horror
2343,0.000385,0.003626,0.005320,0.001732,0.000732,0.001904,0.002478,0.002509,0.020584,0.025593,...,96.0,108.9711,0.215541,-35.0,-21.0,-17.777778,-20.000000,NaN,3.792693,Horror
2344,0.000355,0.003298,0.004922,0.001613,0.000736,0.001921,0.002530,0.002522,0.020116,0.025041,...,96.0,108.9711,0.215541,-35.0,-21.0,-17.777778,-20.000000,NaN,3.792693,Horror
2345,0.000331,0.003107,0.004596,0.001516,0.000747,0.002001,0.002579,0.002548,0.020094,0.024941,...,94.5,112.6776,0.225602,-35.0,-21.0,-17.777778,-20.000000,NaN,2.889626,Horror


In [33]:
from sklearn import metrics
print("Real Accuracy = " + str(metrics.accuracy_score(np.array(output["True Value"]),np.array(output["Prediction"]))*100)+"%")
# print("Simulated Accuracy = " + str(metrics.accuracy_score(np.array(simulate["True Value"]),np.array(simulate["Prediction"]))*100)+"%")

Real Accuracy = 46.71781756180734%
